In [4]:
import pandas as pd
import duckdb as db
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
file = r"C:\Users\USER\Desktop\car_sales_analysis\dataset\cars_database_raw.xlsx"

In [6]:
sheets = pd.read_excel(file,sheet_name=None)
sheets.keys()

dict_keys(['Dealers', 'Models', 'Sales'])

In [7]:
dealers_raw = sheets['Dealers']
dealers_raw

,DealerID,DealerName,City,Country
0,1,Speedster Motors,Kuala Lumpur,Malaysia
1,2,AutoHub Cars,George Town,Malaysia
2,3,Prime Autos,Singapore,Singapore
3,4,Metro Manila Motors,Manila,Philippines
4,5,Visayas Vehicles,Cebu City,Philippines
5,6,Mindanao Motors,Davao City,Philippines
6,7,Capital Cars,Quezon City,Philippines
7,8,Bacolod Wheels,Bacolod City,Philippines
8,9,Iloilo Car Center,Iloilo City,Philippines
9,10,Bangkok Cars,Bangkok,Thailand


In [8]:
dealers_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DealerID    10 non-null     int64 
 1   DealerName  10 non-null     object
 2   City        10 non-null     object
 3   Country     10 non-null     object
dtypes: int64(1), object(3)
memory usage: 452.0+ bytes


In [9]:
dealers_clean = db.sql(
    """
    select
        DealerID::int as dealer_id,
        trim(DealerName)::varchar as dealer_name,
        trim(City)::varchar as city,
        trim(Country)::varchar as country
    from dealers_raw
    """
).df()
dealers_clean

,dealer_id,dealer_name,city,country
0,1,Speedster Motors,Kuala Lumpur,Malaysia
1,2,AutoHub Cars,George Town,Malaysia
2,3,Prime Autos,Singapore,Singapore
3,4,Metro Manila Motors,Manila,Philippines
4,5,Visayas Vehicles,Cebu City,Philippines
5,6,Mindanao Motors,Davao City,Philippines
6,7,Capital Cars,Quezon City,Philippines
7,8,Bacolod Wheels,Bacolod City,Philippines
8,9,Iloilo Car Center,Iloilo City,Philippines
9,10,Bangkok Cars,Bangkok,Thailand


In [10]:
models_raw = sheets['Models']
models_raw

,ModelID,Brand,Model,Segment,EngineSize (L),Fuel,Price (USD),Profit (USD)
0,1,Subaru,Impreza,Sedan,1.5,Gasoline,15000,1500
1,2,Subaru,Forester,SUV,1.8,Gasoline,20000,2000
2,3,Honda,Civic,Sedan,1.8,Gasoline,22000,4400
3,4,Honda,CR-V,SUV,2.4,Gasoline,28000,2800
4,5,Ford,Ranger,Pickup,2.0,Diesel,25000,2500
5,6,Ford,Everest,SUV,2.0,Diesel,30000,3000
6,14,Land Rover,Defender,SUV,1.8,Gasoline,14000,1400
7,15,Land Rover,Range Rover,SUV,3.0,Gasoline,26000,2600
8,9,Mitsubishi,Xpander,MPV,1.5,Gasoline,18000,1800
9,10,Mitsubishi,Outlander,SUV,2.4,Gasoline,32000,3200


In [11]:
models_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ModelID         15 non-null     int64  
 1   Brand           15 non-null     object 
 2   Model           15 non-null     object 
 3   Segment         15 non-null     object 
 4   EngineSize (L)  15 non-null     float64
 5   Fuel            15 non-null     object 
 6   Price (USD)     15 non-null     int64  
 7   Profit (USD)    15 non-null     int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 1.1+ KB


In [12]:
models_clean = db.sql(
    """
    select
        row_number() over() as model_id,
        trim(Model) as model,
        trim(Brand) as brand,
        trim(Segment) as segment,
        "EngineSize (L)"::numeric as engine_size_l,
        trim(Fuel) as fuel_type,
        "Price (USD)"::numeric as unit_price_usd,
        "Profit (USD)"::numeric as unit_profit_usd
    from(
        select distinct(*)
        from models_raw
        order by Brand asc
    )
    order by model_id asc
    """
).df()

In [13]:
models_clean

,model_id,model,brand,segment,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd
0,1,Ranger,Ford,Pickup,2.0,Diesel,25000.0,2500.0
1,2,Everest,Ford,SUV,2.0,Diesel,30000.0,3000.0
2,3,CR-V,Honda,SUV,2.4,Gasoline,28000.0,2800.0
3,4,Civic,Honda,Sedan,1.8,Gasoline,22000.0,4400.0
4,5,Santa Fe,Hyundai,SUV,2.4,Gasoline,34000.0,3400.0
5,6,Accent,Hyundai,Sedan,1.6,Gasoline,16000.0,3200.0
6,7,Mazda3,Mazda,Sedan,2.0,Gasoline,24000.0,2400.0
7,8,Xpander,Mitsubishi,MPV,1.5,Gasoline,18000.0,1800.0
8,9,Outlander,Mitsubishi,SUV,2.4,Gasoline,32000.0,3200.0
9,10,Defender,Land Rover,SUV,1.8,Gasoline,14000.0,1400.0


In [14]:
models_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   model_id         15 non-null     int64  
 1   model            15 non-null     object 
 2   brand            15 non-null     object 
 3   segment          15 non-null     object 
 4   engine_size_l    15 non-null     float64
 5   fuel_type        15 non-null     object 
 6   unit_price_usd   15 non-null     float64
 7   unit_profit_usd  15 non-null     float64
dtypes: float64(3), int64(1), object(4)
memory usage: 1.1+ KB


In [15]:
sales_raw = sheets['Sales']
sales_raw.head()

,SaleID,Date,DealerID,ModelID,Quantity,TotalPrice,Total Profit
0,1,2024-01-05,10,13,1,25000,2500
1,2,2024-01-08,3,5,1,28000,2800
2,3,2024-02-12,8,2,2,28000,2800
3,4,2024-02-22,9,8,1,20000,2000
4,5,2024-03-01,7,4,3,96000,9600


In [16]:
sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   SaleID        218 non-null    int64         
 1   Date          218 non-null    datetime64[ns]
 2   DealerID      218 non-null    int64         
 3   ModelID       218 non-null    int64         
 4   Quantity      218 non-null    int64         
 5   TotalPrice    218 non-null    int64         
 6   Total Profit  218 non-null    int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 12.1 KB


In [17]:
sales_temp_1 = db.sql(
    """
    select
        SaleID as sale_id,
        Date as date,
        DealerID as dealer_id,
        ModelID as model_id_raw,
        Quantity as qty,
        TotalPrice::numeric as total_price_raw,
        "Total Profit"::numeric as total_profit_raw,
        (total_price_raw/qty)::numeric as unit_price_raw,
        (total_profit_raw/qty)::numeric as unit_profit_raw
    from (
        select distinct(*)
        from sales_raw
        order by SaleID asc
    )
    """
).df()
sales_temp_1.head()

,sale_id,date,dealer_id,model_id_raw,qty,total_price_raw,total_profit_raw,unit_price_raw,unit_profit_raw
0,1,2024-01-05,10,13,1,25000.0,2500.0,25000.0,2500.0
1,2,2024-01-08,3,5,1,28000.0,2800.0,28000.0,2800.0
2,3,2024-02-12,8,2,2,28000.0,2800.0,14000.0,1400.0
3,4,2024-02-22,9,8,1,20000.0,2000.0,20000.0,2000.0
4,5,2024-03-01,7,4,3,96000.0,9600.0,32000.0,3200.0


Get sales where `dealer_id` is between 1 and 10


In [18]:
sales_temp_2 = db.sql(
    """
    select *
    from sales_temp_1
    where dealer_id between 1 and 10
    order by sale_id
    """
).df()

In [16]:
sales_temp_2.head()

,sale_id,date,dealer_id,model_id_raw,qty,total_price_raw,total_profit_raw,unit_price_raw,unit_profit_raw
0,1,2024-01-05,10,13,1,25000.0,2500.0,25000.0,2500.0
1,2,2024-01-08,3,5,1,28000.0,2800.0,28000.0,2800.0
2,3,2024-02-12,8,2,2,28000.0,2800.0,14000.0,1400.0
3,4,2024-02-22,9,8,1,20000.0,2000.0,20000.0,2000.0
4,5,2024-03-01,7,4,3,96000.0,9600.0,32000.0,3200.0


Using the `unit_price` and `unit_profit` column, extract the correct `model_id` for each sale from `models_clean` table

In [19]:
sales_temp_3 = db.sql(
    """
    select
        a.sale_id,
        a.date,
        a.dealer_id,
        a.model_id_raw,
        b.model_id as model_id_final,
        a.qty,
        a.total_price_raw,
        a.total_profit_raw,
        a.unit_price_raw,
        a.unit_profit_raw,
        b.unit_price_usd,
        b.unit_profit_usd,
        (b.unit_profit_usd * a.qty)::numeric as total_profit_final
    from sales_temp_2 as a
    left join models_clean as b on a.unit_price_raw = b.unit_price_usd and a.unit_profit_raw = b.unit_profit_usd
    order by sale_id asc
    """
).df()

In [20]:
sales_temp_3.head()

,sale_id,date,dealer_id,model_id_raw,model_id_final,qty,total_price_raw,total_profit_raw,unit_price_raw,unit_profit_raw,unit_price_usd,unit_profit_usd,total_profit_final
0,1,2024-01-05,10,13,1,1,25000.0,2500.0,25000.0,2500.0,25000.0,2500.0,2500.0
1,2,2024-01-08,3,5,3,1,28000.0,2800.0,28000.0,2800.0,28000.0,2800.0,2800.0
2,3,2024-02-12,8,2,10,2,28000.0,2800.0,14000.0,1400.0,14000.0,1400.0,2800.0
3,4,2024-02-22,9,8,13,1,20000.0,2000.0,20000.0,2000.0,20000.0,2000.0,2000.0
4,5,2024-03-01,7,4,9,3,96000.0,9600.0,32000.0,3200.0,32000.0,3200.0,9600.0


In [21]:
car_sales_clean = db.sql(
    """
    select
        sale_id,
        date,
        dealer_id,
        model_id_final as model_id,
        qty as qty_sold
    from sales_temp_3
    """
).df()
car_sales_clean.head()

,sale_id,date,dealer_id,model_id,qty_sold
0,1,2024-01-05,10,1,1
1,2,2024-01-08,3,3,1
2,3,2024-02-12,8,10,2
3,4,2024-02-22,9,13,1
4,5,2024-03-01,7,9,3


In [22]:
car_sales_records = db.sql(
    """
    select
        a.sale_id,
        a.date,
        extract(year from a.date) as date_year,
        extract(month from a.date) as date_month,
        strftime(a.date,'%b') as date_month_name,
        extract(day from a.date) as date_day,
        extract(dow from a.date) as date_dow,
        strftime(a.date,'%a') as date_dow_name,
        c.dealer_name,
        c.country as dealer_country,
        c.city as dealer_city,
        b.model,
        b.segment,
        b.brand,
        b.engine_size_l,
        b.fuel_type,
        b.unit_price_usd,
        b.unit_profit_usd,
        a.qty_sold,
        (a.qty_sold*b.unit_price_usd)::numeric as total_price_usd,
        (a.qty_sold*b.unit_profit_usd)::numeric as total_profit_usd
    from car_sales_clean as a
    left join models_clean as b on b.model_id = a.model_id
    left join dealers_clean as c on c.dealer_id = a.dealer_id
    order by sale_id asc
    """
).df()

In [23]:
car_sales_records.head()

,sale_id,date,date_year,date_month,date_month_name,date_day,date_dow,date_dow_name,dealer_name,dealer_country,...,model,segment,brand,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd,qty_sold,total_price_usd,total_profit_usd
0,1,2024-01-05,2024,1,Jan,5,5,Fri,Bangkok Cars,Thailand,...,Ranger,Pickup,Ford,2.0,Diesel,25000.0,2500.0,1,25000.0,2500.0
1,2,2024-01-08,2024,1,Jan,8,1,Mon,Prime Autos,Singapore,...,CR-V,SUV,Honda,2.4,Gasoline,28000.0,2800.0,1,28000.0,2800.0
2,3,2024-02-12,2024,2,Feb,12,1,Mon,Bacolod Wheels,Philippines,...,Defender,SUV,Land Rover,1.8,Gasoline,14000.0,1400.0,2,28000.0,2800.0
3,4,2024-02-22,2024,2,Feb,22,4,Thu,Iloilo Car Center,Philippines,...,Forester,SUV,Subaru,1.8,Gasoline,20000.0,2000.0,1,20000.0,2000.0
4,5,2024-03-01,2024,3,Mar,1,5,Fri,Capital Cars,Philippines,...,Outlander,SUV,Mitsubishi,2.4,Gasoline,32000.0,3200.0,3,96000.0,9600.0


Save sales records as csv

In [24]:
car_sales_records.to_csv(r"C:\Users\USER\Desktop\car_sales_analysis\dataset\car_sales_records.csv",index=False)

Create a new excel file

In [34]:
with pd.ExcelWriter(r"C:\Users\USER\Desktop\car_sales_analysis\dataset\cars_database_clean.xlsx") as writer:
    dealers_clean.to_excel(writer, sheet_name='dealers_clean', index=False)
    models_clean.to_excel(writer, sheet_name='models_clean', index=False)
    car_sales_clean.to_excel(writer, sheet_name='car_sales_clean', index=False)
    car_sales_records.to_excel(writer, sheet_name='car_sales_records', index=False)

Load clean dimension tables to motherduck for storage

In [25]:
md_token = os.getenv('car_sales_token')

In [31]:
con = db.connect(f'md:?motherduck_token={md_token}')

In [33]:
con.sql("USE car_sales")

In [34]:
con.sql("SELECT current_database()").show()

┌────────────────────┐
│ current_database() │
│      varchar       │
├────────────────────┤
│ car_sales          │
└────────────────────┘



In [ ]:
con.sql(
    """
    create schema if not exists car_sales.dim_facts_records
    """
)

In [37]:
con.execute("CREATE TABLE dim_facts_records.dim_dealers AS SELECT * FROM dealers_clean")

In [38]:
con.execute("CREATE TABLE dim_facts_records.dim_models AS SELECT * FROM models_clean")

In [39]:
con.execute("CREATE TABLE dim_facts_records.car_sales_clean AS SELECT * FROM car_sales_clean")

In [42]:
con.execute(
    """
    create table dim_facts_records.car_sales_records as
    select
        a.sale_id,
        a.date,
        extract(year from a.date) as date_year,
        extract(month from a.date) as date_month,
        strftime(a.date,'%b') as date_month_name,
        extract(day from a.date) as date_day,
        extract(dow from a.date) as date_dow,
        strftime(a.date,'%a') as date_dow_name,
        c.dealer_name,
        c.country as dealer_country,
        c.city as dealer_city,
        b.model,
        b.segment,
        b.brand,
        b.engine_size_l,
        b.fuel_type,
        b.unit_price_usd,
        b.unit_profit_usd,
        a.qty_sold,
        (a.qty_sold*b.unit_price_usd)::numeric as total_price_usd,
        (a.qty_sold*b.unit_profit_usd)::numeric as total_profit_usd
    from dim_facts_records.car_sales_clean as a
    left join dim_facts_records.dim_models as b on b.model_id = a.model_id
    left join dim_facts_records.dim_dealers as c on c.dealer_id = a.dealer_id
    order by a.sale_id asc
    """
)

In [47]:
con.sql(
    """
    select *
    from dim_facts_records.car_sales_records
    limit 10
    
    """
).show()

┌─────────┬─────────────────────┬───────────┬────────────┬─────────────────┬──────────┬──────────┬───────────────┬─────────────────────┬────────────────┬──────────────┬───────────┬─────────┬────────────┬───────────────┬───────────┬────────────────┬─────────────────┬──────────┬─────────────────┬──────────────────┐
│ sale_id │        date         │ date_year │ date_month │ date_month_name │ date_day │ date_dow │ date_dow_name │     dealer_name     │ dealer_country │ dealer_city  │   model   │ segment │   brand    │ engine_size_l │ fuel_type │ unit_price_usd │ unit_profit_usd │ qty_sold │ total_price_usd │ total_profit_usd │
│  int64  │    timestamp_ns     │   int64   │   int64    │     varchar     │  int64   │  int64   │    varchar    │       varchar       │    varchar     │   varchar    │  varchar  │ varchar │  varchar   │    double     │  varchar  │     double     │     double      │  int64   │  decimal(18,3)  │  decimal(18,3)   │
├─────────┼─────────────────────┼───────────┼──────────